# pydantic with gemini

In [ ]:
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
response = client.models.generate_content(model="gemini-2.5-flash", contents="tell me a programming joke")
print(response.text)


In [ ]:
response = client.models.generate_content(model="gemini-2.5-flash", contents="""
you are a helpful assistant. I need you to create a JSON object representing a library.
The library's name should be "Coolu Library" and have the fields name and books that contains a list of books.
Each book should have a 'title', 'author' and 'year' field. Make sure the output is a single, valid JSON object.
Give me 10 books.
""")

response.text

In [ ]:
print(response.text)

### pydantic to validate simulated data

In [ ]:
from pydantic import BaseModel, Field
from datetime import datetime

class Book(BaseModel):
    title: str
    author: str
    year: int = Field(gt = 1000, lt= datetime.now().year)
    
class Library(BaseModel):
    name: str
    books: list[Book]

library = Library.model_validate_json(response.text)
    

In [ ]:
library.name

In [ ]:
library.books

In [ ]:
library.books[1].title, library.books[1].year

In [ ]:
titles = [books.title for books in library.books]
titles

In [ ]:
newer_books = [(book.title, book.year) for book in library.books if book.year > 1970]
newer_books

In [ ]:
library.model_dump()

In [ ]:
library.model_dump_json()

In [ ]:
with open("library.json", "w") as json_file:
    json_file.write(library).model_dump_json(indent=4)

### create pandas dataframe

In [ ]:
import pandas as pd

titles = [book.title for book in library.books]
authors = [book.author for book in library.books]
years = [book.year for book in library.books]

df = pd.DataFrame({"title":titles, "year":years, "author":authors})
df

In [ ]:
df["year"].plot(kind = "hist", bins=20)